# EGNN Molecular Represitation

**Licenses**

Copyright (c) DP Technology.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.


In [1]:
!pip install lmdb
!pip install rdkit
!pip install https://github.com/dptech-corp/Uni-Core/releases/download/0.0.1/unicore-0.0.1+cu113torch1.12.1-cp39-cp39-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.5/298.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 43.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.4 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached protobuf-3.20.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94506 sha256=1f17d187360d0dfc4941a0411b9b8268633a999f4d1d1057340d7af81dfef97c
  Stored in directory: /home/amit/.cache/pip/wheels/fd/c2/0d/5d94d95e5875ea17b85a9f1f99b8dd2e50517137c8042c6468
Successfully built ml-collections
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.0
    Uninstalling protobuf-4.23.

In [1]:
import os
import numpy as np
import pandas as pd
import lmdb
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
import pickle
import glob

### Your SMILES list

In [2]:
smi_list = [
'C1C2C(C3C(C(O2)O)OC(=O)C4=CC(=C(C(=C4C5=C(C(=C(C=C5C(=O)O3)O)O)O)O)O)O)OC(=O)C6=CC(=C(C(=C6C7=C(C(=C8C9=C7C(=O)OC2=C(C(=C(C3=C(C(=C(C=C3C1=O)O)O)O)C(=C92)C(=O)O8)O)O)O)O)O)O)O',
'C1C2C(C3C(C(O2)OC(=O)C4=CC(=C(C(=C4)OC5=C(C(=C(C6=C5C(=O)OC7C(COC(=O)C8=CC(=C(C(=C86)O)O)O)OC(C9C7OC(=O)C2=CC(=C(C(=C2C2=C(C(=C(C=C2C(=O)O9)O)O)O)O)O)O)OC(=O)C2=CC(=C(C(=C2)OC2=C(C(=C(C4=C2C(=O)OC2C(COC(=O)C5=CC(=C(C(=C54)O)O)O)OC(C4C2OC(=O)C2=CC(=C(C(=C2C2=C(C(=C(C=C2C(=O)O4)O)O)O)O)O)O)OC(=O)C2=CC(=C(C(=C2)OC2=C(C(=C(C4=C2C(=O)OC2C(COC(=O)C5=CC(=C(C(=C54)O)O)O)OC(C4C2OC(=O)C2=CC(=C(C(=C2C2=C(C(=C(C=C2C(=O)O4)O)O)O)O)O)O)OC(=O)C2=CC(=C(C(=C2)O)O)O)O)O)O)O)O)O)O)O)O)O)O)O)O)O)O)OC(=O)C2=CC(=C(C(=C2C2=C(C(=C(C=C2C(=O)O3)O)O)O)O)O)O)OC(=O)C2=CC(=C(C(=C2C2=C(C(=C(C=C2C(=O)O1)O)O)O)O)O)O',
'C1=CC2=C(C=CN=C2)C(=C1)S(=O)(=O)NCCNCCC(=O)NC(CO)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)O'
]

In [11]:

df = pd.read_csv('/data/amit/DAVIS/KIBA_with_Emb/unique_smiles.csv')

# Extract the SMILES column as a list
smile_list = df['Drug'].tolist()

len(smile_list)

2065

In [18]:
import json

file_path = "/data/amit/DAVIS/ligands_can_kiba.txt"

with open(file_path, "r") as file:
    data = json.load(file)



In [69]:
#Update the bad conformer ID

key_to_update = "CHEMBL436718"  # Key whose value you want to change
new_value = "N=C(N)NCCC[C@H](NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CO)NC(=O)CCNCCNS(=O)(=O)c1cccc2cnccc12)C(=O)O"  # New value to assign to the key

updated_data[key_to_update] = new_value  # Update the value associated with the key

# Return the updated dictionary (optional)
#updated_data = data

    
smile_list = list(updated_data.values())
len(smile_list)

2111

In [36]:
smile_list[1864]


'C1=CC2=C(C=CN=C2)C(=C1)S(=O)(=O)NCCNCCC(=O)NC(CO)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)O'

### Generate conformations from SMILES and save to .lmdb

In [20]:
def smi2coords(smi, seed):
    mol = Chem.MolFromSmiles(smi)
    mol = AllChem.AddHs(mol)
    atoms = [atom.GetSymbol() for atom in mol.GetAtoms()]
    coordinate_list = []
    res = AllChem.EmbedMolecule(mol, randomSeed=seed)
    if res == 0:
        try:
            AllChem.MMFFOptimizeMolecule(mol)
        except:
            pass
        coordinates = mol.GetConformer().GetPositions()
    elif res == -1:
        mol_tmp = Chem.MolFromSmiles(smi)
        AllChem.EmbedMolecule(mol_tmp, maxAttempts=5000, randomSeed=seed)
        mol_tmp = AllChem.AddHs(mol_tmp, addCoords=True)
        try:
            AllChem.MMFFOptimizeMolecule(mol_tmp)
        except:
            pass
        coordinates = mol_tmp.GetConformer().GetPositions()
    assert len(atoms) == len(coordinates), "coordinates shape is not align with {}".format(smi)
    coordinate_list.append(coordinates.astype(np.float32))
    return pickle.dumps({'atoms': atoms, 'coordinates': coordinate_list, 'smi': smi}, protocol=-1)

def write_lmdb(smiles_list, job_name, seed=42, outpath='./results'):
    os.makedirs(outpath, exist_ok=True)
    output_name = os.path.join(outpath,'{}.lmdb'.format(job_name))
    try:
        os.remove(output_name)
    except:
        pass
    env_new = lmdb.open(
        output_name,
        subdir=False,
        readonly=False,
        lock=False,
        readahead=False,
        meminit=False,
        max_readers=1,
        map_size=int(100e9),
    )
    txn_write = env_new.begin(write=True)
    for i, smiles in tqdm(enumerate(smiles_list)):
        inner_output = smi2coords(smiles, seed=seed)
        txn_write.put(f"{i}".encode("ascii"), inner_output)
    txn_write.commit()
    env_new.close()

In [66]:
smile_list[1864]

'C1=CC2=C(C=CN=C2)C(=C1)S(=O)(=O)NCCNCCC(=O)NC(CO)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)O'

C1=CC2=C(C=CN=C2)C(=C1)S(=O(=O)NCCNCCC(=O)NC(CO)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)NC(CCCN=C(N)N)C(=O)O

In [17]:
print("Smile for ID 1799:", smile_list[1799])
#print("Smile for ID 1367:", smile_list[1367])

Smile for ID 1799: C1C2C(C3C(C(O2)O)OC(=O)C4=CC(=C(C(=C4C5=C(C(=C(C=C5C(=O)O3)O)O)O)O)O)O)OC(=O)C6=CC(=C(C(=C6C7=C(C(=C8C9=C7C(=O)OC2=C(C(=C(C3=C(C(=C(C=C3C1=O)O)O)O)C(=C92)C(=O)O8)O)O)O)O)O)O)O


In [57]:
# assuming your list is named 'my_list'
smile_list2 = smile_list[1864:]
#1799

In [68]:
smiles = 'N=C(N)NCCC[C@H](NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@H](CO)NC(=O)CCNCCNS(=O)(=O)c1cccc2cnccc12)C(=O)O'
inner_output = smi2coords(smiles, seed=42)


[15:06:21] Molecule does not have explicit Hs. Consider calling AddHs()


In [18]:
smile_list3 = smile_list2[5:]

In [23]:
smile_list4 = smile_list3[24:]

In [70]:
seed = 42
job_name = 'get_mol_repr'   #custom name
data_path = '/data/amit/results'  #data path
weight_path='/home/amit/Uni-Mol/unimol/mol_pre_no_h_220816.pt'  #ckpt path
only_polar=0  # no h
dict_name='dict.txt'
batch_size=16
results_path=data_path   #save path
write_lmdb(smile_list, job_name=job_name, seed=seed, outpath=data_path)

1693it [02:47, 10.10it/s][15:10:19] Molecule does not have explicit Hs. Consider calling AddHs()
[15:10:20] Molecule does not have explicit Hs. Consider calling AddHs()
1699it [02:50,  9.97it/s][15:10:22] Molecule does not have explicit Hs. Consider calling AddHs()
1819it [03:17,  9.23it/s][15:10:49] Molecule does not have explicit Hs. Consider calling AddHs()
1837it [03:26,  8.87it/s][15:10:57] Molecule does not have explicit Hs. Consider calling AddHs()
1843it [03:34,  8.61it/s][15:11:05] Molecule does not have explicit Hs. Consider calling AddHs()
1861it [03:41,  8.41it/s][15:11:13] Molecule does not have explicit Hs. Consider calling AddHs()
1873it [03:47,  8.24it/s][15:11:18] Molecule does not have explicit Hs. Consider calling AddHs()


### Infer from ckpt

In [71]:
# NOTE: Currently, the inference is only supported to run on a single GPU. You can add CUDA_VISIBLE_DEVICES="0" before the command.
!cp ../example_data/molecule/$dict_name $data_path
#!TORCH_USE_RTLD_GLOBAL=1
!CUDA_VISIBLE_DEVICES="0" python ../unimol/infer.py --user-dir ../unimol $data_path --valid-subset $job_name \
       --results-path $results_path \
       --num-workers 8 --ddp-backend=c10d --batch-size $batch_size \
       --task unimol --loss unimol_infer --arch unimol_base \
       --path $weight_path \
       --fp16-init-scale 4 --fp16-scale-window 256 \
       --only-polar $only_polar --dict-name $dict_name \
       --log-interval 50 --log-format simple --random-token-prob 0 --leave-unmasked-prob 1.0 --mode infer


fused_multi_tensor is not installed corrected
fused_rounding is not installed corrected
fused_layer_norm is not installed corrected
fused_softmax is not installed corrected
2023-06-15 15:12:05 | INFO | unimol.inference | loading model(s) from /home/amit/Uni-Mol/unimol/mol_pre_no_h_220816.pt
2023-06-15 15:12:06 | INFO | unimol.tasks.unimol | dictionary: 30 types
2023-06-15 15:12:09 | INFO | unimol.inference | Namespace(no_progress_bar=False, log_interval=50, log_format='simple', tensorboard_logdir='', seed=1, cpu=False, fp16=False, bf16=False, bf16_sr=False, allreduce_fp32_grad=False, fp16_no_flatten_grads=False, fp16_init_scale=4, fp16_scale_window=256, fp16_scale_tolerance=0.0, min_loss_scale=0.0001, threshold_loss_scale=None, user_dir='../unimol', empty_cache_freq=0, all_gather_list_size=16384, suppress_crashes=False, profile=False, ema_decay=-1.0, loss='unimol_infer', optimizer='adam', lr_scheduler='fixed', task='unimol', num_workers=8, skip_invalid_size_inputs_valid_test=False, bat

In [ ]:
# NOTE: Currently, the inference is only supported to run on a single GPU. You can add CUDA_VISIBLE_DEVICES="0" before the command.
!cp ../example_data/molecule/$dict_name $data_path
#!TORCH_USE_RTLD_GLOBAL=1
!CUDA_VISIBLE_DEVICES="2" python ../unimol/infer.py --user-dir ../unimol $data_path --valid-subset $job_name \
       --results-path $results_path \
       --num-workers 8 --ddp-backend=c10d --batch-size $batch_size \
       --task unimol --loss unimol_infer --arch unimol_base \
       --path $weight_path \
       --fp16 --fp16-init-scale 4 --fp16-scale-window 256 \
       --only-polar $only_polar --dict-name $dict_name \
       --log-interval 50 --log-format simple --random-token-prob 0 --leave-unmasked-prob 1.0 --mode infer

### Read .pkl and save results to .csv

In [ ]:
def get_csv_results(predict_path, results_path):
    predict = pd.read_pickle(predict_path)
    smi_list, mol_repr_list, pair_repr_list = [], [], []
    for batch in predict:
        sz = batch["bsz"]
        for i in range(sz):
            smi_list.append(batch["smi_name"][i])
            mol_repr_list.append(batch["mol_repr_cls"][i])
            pair_repr_list.append(batch["pair_repr"][i])
    predict_df = pd.DataFrame({"SMILES": smi_list, "mol_repr": mol_repr_list, "pair_repr": pair_repr_list})
    print(predict_df.head(1),predict_df.info())
    predict_df.to_csv(results_path+'/mol_repr.csv',index=False)

pkl_path = glob.glob(f'{results_path}/*_{job_name}.out.pkl')[0]
get_csv_results(pkl_path, results_path)

In [72]:
import numpy as np

# Save the embeddings in a .npy file
def get_mol_repr(predict_path, result_path):
    predict = pd.read_pickle(predict_path)
    mol_repr_list = []
    for batch in predict:
        sz = batch["bsz"]
        for i in range(sz):
            mol_repr_list.append(batch["mol_repr_cls"][i])
    mol_repr_arr = np.array(mol_repr_list)
    np.save(f"{result_path}/mol_repr_arr.npy", mol_repr_arr)
    
pkl_path = glob.glob(f'{results_path}/*_{job_name}.out.pkl')[0]
get_mol_repr(pkl_path, results_path)

In [73]:
emb = np.load('/data/amit/results/mol_repr_arr.npy')
emb.shape

(2111, 512)